In [1]:
import os
os.chdir("../")
# os.chdir("src")
%ls

 Volume in drive C is Windows
 Volume Serial Number is 666A-2AE9

 Directory of c:\Users\Nilesh\Documents\GitHub\End to End ClearSight AI

17-04-2024  20:13    <DIR>          .
17-04-2024  12:13    <DIR>          ..
17-04-2024  12:17    <DIR>          .github
17-04-2024  16:30             3,251 .gitignore
17-04-2024  18:35    <DIR>          artifacts
17-04-2024  12:17    <DIR>          config
17-04-2024  12:17                 0 dvc.yaml
17-04-2024  12:47    <DIR>          logs
17-04-2024  18:18             1,377 main.py
17-04-2024  15:11               168 params.yaml
17-04-2024  20:09               219 README.md
17-04-2024  12:40               194 requirements.txt
17-04-2024  20:21    <DIR>          research
17-04-2024  12:37             1,066 setup.py
17-04-2024  14:01    <DIR>          src
17-04-2024  20:13                 9 tempCodeRunnerFile.py
17-04-2024  12:17             1,327 template.py
17-04-2024  12:17    <DIR>          templates
               9 File(s)          7,611 bytes

In [2]:

os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/nileshparab5623/End-to-End-ClearSightAI.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="nileshparab5623"
os.environ["MLFLOW_TRACKING_PASSWORD"]="129584ae8da4b65589cd989d7119f93455a995c3"

In [3]:
import tensorflow as tf
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [4]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',  # Use binary cross-entropy for binary classification
              metrics=['accuracy'])

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [6]:
os.chdir("src")

In [7]:
from clearSightAI.constants import *
from clearSightAI.utils.common import read_yaml, create_directories, save_json

In [8]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/CIFAR1000",
            mlflow_uri="https://dagshub.com/nileshparab5623/End-to-End-ClearSightAI.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [9]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [10]:
os.chdir("../")
%ls

 Volume in drive C is Windows
 Volume Serial Number is 666A-2AE9

 Directory of c:\Users\Nilesh\Documents\GitHub\End to End ClearSight AI

17-04-2024  20:13    <DIR>          .
17-04-2024  12:13    <DIR>          ..
17-04-2024  12:17    <DIR>          .github
17-04-2024  16:30             3,251 .gitignore
17-04-2024  18:35    <DIR>          artifacts
17-04-2024  12:17    <DIR>          config
17-04-2024  12:17                 0 dvc.yaml
17-04-2024  12:47    <DIR>          logs
17-04-2024  18:18             1,377 main.py
17-04-2024  15:11               168 params.yaml
17-04-2024  20:09               219 README.md
17-04-2024  12:40               194 requirements.txt
17-04-2024  20:21    <DIR>          research
17-04-2024  12:37             1,066 setup.py
17-04-2024  14:01    <DIR>          src
17-04-2024  20:13                 9 tempCodeRunnerFile.py
17-04-2024  12:17             1,327 template.py
17-04-2024  12:17    <DIR>          templates
               9 File(s)          7,611 bytes

In [11]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")


In [12]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e



Found 300 images belonging to 2 classes.


c:\Users\Nilesh\miniconda3\envs\csenv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


19/19 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - accuracy: 0.1877 - loss: 0.6973


2024/04/17 20:50:29 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
Successfully registered model 'VGG16Model'.
2024/04/17 20:53:37 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.
